<a href="https://colab.research.google.com/github/shubhamsks/deep-learning/blob/master/cnn_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In  this cell we will make Convulation Layer using tensor flow to take the image of dimesnion as specified  as input

In [0]:
import tensorflow as tf
# depth of the output
k_output = 64

# image dimensions 
image_width = 10
image_height = 10
color_channels = 3
# convolution filter dimension
filter_size_width = 5
filter_size_height = 5
# input /image
input = tf.placeholder(tf.float32, shape = [None, image_height, image_width, color_channels])

In [0]:
# weight and bias
weight = tf.Variable(tf.truncated_normal([filter_size_height, filter_size_width,color_channels, k_output]))

bias = tf.Variable(tf.zeros(k_output))

In [0]:
# apply convolution 
conv_layer = tf.nn.conv2d(input, weight,strides = [1,2,2,1], padding = 'SAME')
# add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)


TensorFlow uses a ```stride``` for each input dimension, 

```[batch, input_height, input_width, input_channels]```


We generally always set the stride for batch and input_channels (i.e. the first and fourth element in the strides array) to be 1. This ensures that the model uses all batches and input channels. (It's good practice to remove the batches or channels you want to skip from the data set rather than use a stride to skip them.)

In [8]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)
print(X.shape)


(1, 4, 4, 1)


In [0]:


def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.truncated_normal([2,2,1,3]) )
    F_b = tf.Variable(tf.zeros(3))
    strides = [1,2,2,1]
    padding = 'VALID'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

out = conv2d(X)


In [11]:
out

<tf.Tensor 'add:0' shape=(1, 2, 2, 3) dtype=float32>

In [0]:
# Max pooling using tf
conv_layer = out
conv_layer = tf.nn.relu(conv_layer)
# apply max pool
conv_layer = tf.nn.max_pool(conv_layer, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [0]:
"""
Set the values to `strides` and `ksize` such that
the output shape after pooling is (1, 2, 2, 1).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.max_pool` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    # TODO: Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
    ksize = [1,2,2,1]
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1,2,2,1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#max_pool
    return tf.nn.max_pool(input, ksize, strides, padding)
    
out = maxpool(X)